In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
df = pd.read_csv("../data/processed/predictions.csv")
df.head()


,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionID,actual,predicted,fraud_probability
0,7864752,107.950,4,7585,553.0,150.0,4,226.0,1,264.0,...,2,2,2,2,1,1735,3303078,0,0,0.101259
1,2260318,37.021,0,3154,408.0,185.0,2,224.0,2,-999.0,...,0,0,1,1,0,1598,3103888,0,0,0.234933
2,10361225,59.000,4,10057,225.0,150.0,2,224.0,2,181.0,...,2,2,2,2,1,1735,3397211,0,0,0.210129
3,5974846,57.950,4,6207,355.0,150.0,4,166.0,2,143.0,...,2,2,2,2,1,1735,3238027,0,0,0.078260
4,4296816,107.950,4,11207,361.0,150.0,4,226.0,2,325.0,...,2,2,2,2,1,1735,3178265,0,0,0.081347


In [3]:
engine = create_engine("sqlite:///../fraud.db") 

In [4]:
df.to_sql("fraud_scores", con=engine, if_exists="replace", index=False)

118108